In [21]:
import numpy as np
import copy

In [22]:
def sigmoid(x):
    return 1 / (1+ np.exp(-x)) 

In [23]:
def derivativeSigmoidx(x):  #slope
    return x * (1 - x)

In [24]:
binaryDim = 8
largest_num = pow(2,binaryDim)

In [25]:
largest_num


256

In [26]:
int_2_bin = {}

In [38]:
binary = np.unpackbits(np.array([range(largest_num)], dtype=np.uint8).T, axis=1)  #uint= unsigned int

In [39]:
for i in range(largest_num):
    int_2_bin[i] = binary[i]

In [40]:
int_2_bin

{0: array([0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 1: array([0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8),
 2: array([0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8),
 3: array([0, 0, 0, 0, 0, 0, 1, 1], dtype=uint8),
 4: array([0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8),
 5: array([0, 0, 0, 0, 0, 1, 0, 1], dtype=uint8),
 6: array([0, 0, 0, 0, 0, 1, 1, 0], dtype=uint8),
 7: array([0, 0, 0, 0, 0, 1, 1, 1], dtype=uint8),
 8: array([0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8),
 9: array([0, 0, 0, 0, 1, 0, 0, 1], dtype=uint8),
 10: array([0, 0, 0, 0, 1, 0, 1, 0], dtype=uint8),
 11: array([0, 0, 0, 0, 1, 0, 1, 1], dtype=uint8),
 12: array([0, 0, 0, 0, 1, 1, 0, 0], dtype=uint8),
 13: array([0, 0, 0, 0, 1, 1, 0, 1], dtype=uint8),
 14: array([0, 0, 0, 0, 1, 1, 1, 0], dtype=uint8),
 15: array([0, 0, 0, 0, 1, 1, 1, 1], dtype=uint8),
 16: array([0, 0, 0, 1, 0, 0, 0, 0], dtype=uint8),
 17: array([0, 0, 0, 1, 0, 0, 0, 1], dtype=uint8),
 18: array([0, 0, 0, 1, 0, 0, 1, 0], dtype=uint8),
 19: array([0, 0, 0, 1, 0, 0, 1, 1], dtyp

In [47]:
alpha = 0.1
inpurNeurons = 2
hiddenNeurons = 16
outputNeurons = 1

In [48]:
weights_hidden = np.random.random((inpurNeurons, hiddenNeurons))
weights_output = np.random.random((hiddenNeurons, outputNeurons))
weights_time = np.random.random((hiddenNeurons, hiddenNeurons))

In [49]:
weights_hidden_update = np.zeros_like(weights_hidden)
weights_output_update = np.zeros_like(weights_output)
weights_time_update = np.zeros_like(weights_time)

In [53]:
epochs = 10000

for i in range(epochs):
    a_int = np.random.randint(largest_num//2)
    b_int = np.random.randint(largest_num//2)
    
    a_bin = int_2_bin[a_int]
    b_bin = int_2_bin[b_int]
    
    c_int = a_int + b_int
    c_bin = int_2_bin[c_int]
    
    d = np.zeros_like(c_bin)
    overallError = 0
    
    layer_1_values = list()
    layer_2_deltas = list()
    layer_1_values.append(np.zeros(hiddenNeurons))
    
#  2: array([0, 0, 0, 0, 0, 0, 1, 0])
#  3: array([0, 0, 0, 0, 0, 0, 1, 1])
#  5: array([0, 0, 0, 0, 0, 1, 0, 1])
    
    for position in range(binaryDim):
        X = np.array([[a_bin[binaryDim - position - 1], b_bin[binaryDim - position - 1]]])
        y = np.array([[c_bin[binaryDim - position - 1]]])
        
        hidden_layer = sigmoid(np.dot(X, weights_hidden) + np.dot(layer_1_values[-1], weights_time))
        output_layer = sigmoid(np.dot(hidden_layer, weights_output))
        
        output_layer_error = y - output_layer
        layer_2_deltas.append((output_layer_error) * derivativeSigmoidx(output_layer))
        
        overallError += np.abs(output_layer_error[0])
        
        d[binaryDim - position - 1] = np.round(output_layer[0][0])
        layer_1_values.append(copy.deepcopy(hidden_layer))
        
    future_layer_delta = np.zeros(hiddenNeurons)
    
    for position in range(binaryDim):
        X = np.array([[a_bin[position], b_bin[position]]])
        hidden_layer = layer_1_values[-position - 1]
        prev_hidden_layer = layer_1_values[-position - 2]
        
        output_layer_delta = layer_2_deltas[-position - 1]
        hidden_layer_delta = future_layer_delta.dot(weights_time.T) + output_layer_delta.dot(weights_output.T) * derivativeSigmoidx(hidden_layer)
        
        weights_output_update += np.atleast_2d(hidden_layer).T.dot(output_layer_delta)
        weights_hidden_update += X.T.dot(hidden_layer_delta)
        weights_time_update += np.atleast_2d(prev_hidden_layer).T.dot(hidden_layer_delta)
        
        future_layer_delta = hidden_layer_delta
    
    weights_hidden += weights_hidden_update * alpha
    weights_output += weights_output_update * alpha
    weights_time += weights_time_update * alpha
    
    if (i % 100 == 0):
        print("Overall Error :",overallError)
        print("Pred :",d)
        print("Actual :",c_bin)
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2, index)
        print("{} + {} = {}".format(a_int, b_int, c_int))
        print("{} + {} = {}".format(a_int, b_int, out))
        print("*" * 100)

Overall Error : [4.99554703]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 0 1 0 1 0 1 0]
15 + 27 = 42
15 + 27 = 255
****************************************************************************************************


<ipython-input-22-615268fff036>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1+ np.exp(-x))


Overall Error : [4.96432816]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 0 1 1 0 0 0 1]
17 + 32 = 49
17 + 32 = 255
****************************************************************************************************
Overall Error : [5.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 0 0 1 0 1 0]
65 + 9 = 74
65 + 9 = 255
****************************************************************************************************
Overall Error : [6.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 0 0 0 0 0 1]
17 + 112 = 129
17 + 112 = 255
****************************************************************************************************
Overall Error : [6.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 0 0 0 1 0 0]
18 + 114 = 132
18 + 114 = 255
****************************************************************************************************
Overall Error : [3.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 1 0 1 0 1 1]
81 + 90 = 171
81 + 90 = 255
***********************************************************************************

Overall Error : [1.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 1 0 1 1 1 1 1]
100 + 123 = 223
100 + 123 = 255
****************************************************************************************************
Overall Error : [5.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 1 0 1 0 0 0]
3 + 101 = 104
3 + 101 = 255
****************************************************************************************************
Overall Error : [1.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 1 1 1 1 1 1]
65 + 62 = 127
65 + 62 = 255
****************************************************************************************************
Overall Error : [4.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 1 1 1 0 0 0]
35 + 85 = 120
35 + 85 = 255
****************************************************************************************************
Overall Error : [2.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 0 1 1 1 1 1]
106 + 53 = 159
106 + 53 = 255
*************************************************************************************

Overall Error : [2.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 1 1 1 1 1 0]
70 + 56 = 126
70 + 56 = 255
****************************************************************************************************
Overall Error : [4.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 0 1 0 0 1 1]
96 + 51 = 147
96 + 51 = 255
****************************************************************************************************
Overall Error : [5.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 1 0 0 1 0 1 0]
31 + 43 = 74
31 + 43 = 255
****************************************************************************************************
Overall Error : [5.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [1 0 0 0 0 1 0 1]
105 + 28 = 133
105 + 28 = 255
****************************************************************************************************
Overall Error : [6.]
Pred : [1 1 1 1 1 1 1 1]
Actual : [0 0 1 0 0 0 0 1]
11 + 22 = 33
11 + 22 = 255
*******************************************************************************************